In [1]:
!nvidia-smi

Sun Nov 21 13:03:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.67       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3090    Off  | 00000000:65:00.0 Off |                  N/A |
|  0%   50C    P8    48W / 370W |   1581MiB / 24265MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 3090    Off  | 00000000:B3:00.0 Off |                  N/A |
|  0%   

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import skimage.transform as st
import os
import gc
import warnings
from sklearn.metrics import classification_report, average_precision_score
from Parse_TFrecords import *
from define_model import *
from load_data import *
from utilities import *
 
print(tf.__version__)

warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

gpus = tf.config.list_physical_devices(device_type='GPU')
tf.config.set_visible_devices(devices=gpus[1], device_type='GPU')

2.5.0


In [3]:
pos = 27297
neg = 119133

total = pos+neg

weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 0.61
Weight for class 1: 2.68


In [4]:
IMG_SIZE = 256

# Create a generator

data_augmentation_1 = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomRotation((-0.01, 0.01), fill_mode='reflect')
])

data_augmentation_2 = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomRotation((-0.005, 0.005), fill_mode='reflect')
])

## Augmentation

In [ ]:
BATCH_SIZE = 32
checkpoint_filepath = 'checkpoints/PrecisionAtRecall/checkpoint_aug'
archi = 'Dnet121'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    save_weights_only=True,
    monitor='val_precision_at_recall',
    mode='max')
    
callback = [tf.keras.callbacks.LearningRateScheduler(scheduler),
            tf.keras.callbacks.EarlyStopping(mode='max', patience=3, monitor='val_precision_at_recall'),
            model_checkpoint_callback]

record_file_train = 'copd_train_new.tfrecords'
train_majority = (tf.data.TFRecordDataset(
    record_file_train, compression_type=None, buffer_size=32, num_parallel_reads=32)
.map(parse_TFrecord_train)
.filter(filter_0)) # filter out negative cases
                  
train_minority = (tf.data.TFRecordDataset(
    record_file_train, compression_type=None, buffer_size=32, num_parallel_reads=32)
.map(parse_TFrecord_train)
.filter(filter_1)) # filter out positive cases

train_majority = train_majority.batch(1).map(lambda x, y: (data_augmentation_1(x, training=True), y), 
                num_parallel_calls=tf.data.AUTOTUNE).unbatch()

train_majority = train_majority.concatenate(train_minority.batch(1).map(lambda x, y: (data_augmentation_1(x, training=True), y), 
                num_parallel_calls=tf.data.AUTOTUNE).unbatch())

train_dataset = train_majority.concatenate(train_minority.batch(1).map(lambda x, y: (data_augmentation_2(x, training=True), y), 
                num_parallel_calls=tf.data.AUTOTUNE).unbatch())

train_dataset = train_dataset.shuffle(total).batch(BATCH_SIZE)

record_file_val = 'copd_val_new.tfrecords'
val_majority = (tf.data.TFRecordDataset(
    record_file_val, compression_type=None, buffer_size=32, num_parallel_reads=32)
.map(parse_TFrecord_train)
.filter(filter_0)) # filter out negative cases

val_minority = (tf.data.TFRecordDataset(
    record_file_val, compression_type=None, buffer_size=32, num_parallel_reads=32)
.map(parse_TFrecord_train)
.filter(filter_1)) # filter out positive cases

val_majority = val_majority.batch(1).map(lambda x, y: (data_augmentation_1(x, training=True), y), 
                num_parallel_calls=tf.data.AUTOTUNE).unbatch()

val_majority = val_majority.concatenate(val_minority.batch(1).map(lambda x, y: (data_augmentation_1(x, training=True), y), 
                num_parallel_calls=tf.data.AUTOTUNE).unbatch())

val_dataset = train_majority.concatenate(val_minority.batch(1).map(lambda x, y: (data_augmentation_2(x, training=True), y), 
                num_parallel_calls=tf.data.AUTOTUNE).unbatch())

val_dataset = val_dataset.shuffle(36601).batch(BATCH_SIZE)

model = load_model_from_pretrain(archi)

metric = tf.keras.metrics.PrecisionAtRecall(recall=0.5, name='precision_at_recall')

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                 optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=metric)

model.fit(train_dataset, epochs=20, shuffle=True, validation_data=val_dataset, callbacks=callback)

gc.collect()

Epoch 1/20
   5429/Unknown - 891s 148ms/step - loss: 0.5555 - precision_at_recall: 0.5272

In [5]:
checkpoint_filepath = 'checkpoints/PrecisionAtRecall/checkpoint_aug'
archi = 'Dnet121'

model = define_model(archi)

model.load_weights(checkpoint_filepath)

In [16]:
record_file = 'copd_test_new.tfrecords'
test_majority = (tf.data.TFRecordDataset(
    record_file, compression_type=None, buffer_size=32, num_parallel_reads=32)
.map(parse_TFrecord_train)
.filter(filter_0)
.batch(32)) # filter out negative cases

test_minority = (tf.data.TFRecordDataset(
    record_file, compression_type=None, buffer_size=32, num_parallel_reads=32)
.map(parse_TFrecord_train)
.filter(filter_1)
.batch(32)) # filter out positive cases

test_majority = test_majority.map(lambda x, y: (data_augmentation(x, training=True), y), 
                num_parallel_calls=tf.data.AUTOTUNE)

test_minority = test_minority.concatenate(test_minority.map(lambda x, y: (data_augmentation(x, training=True), y), 
                num_parallel_calls=tf.data.AUTOTUNE))

y_preds_minority = []
y_preds = model.predict(test_minority)
length = int(len(y_preds)/2)
for i in range(length):
    y_preds_minority.append((y_preds[i]+y_preds[length+i])/2)

y_preds_minority = np.array(y_preds_minority)
y_preds_majority = model.predict(test_majority)

In [17]:
y_ones = np.ones((length, 1))
y_zeros = np.zeros(y_preds_majority.shape)

y_test = np.append(y_ones, y_zeros)
y_preds = np.append(y_preds_minority, y_preds_majority)

test_CI(y_preds, y_test)

thresh = get_thresh(y_test, y_preds, 'Youden')

print(thresh)
print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))
print(average_precision_score(y_test, y_preds, average=None))

gc.collect()

Sample auc mean: 0.91
Samole auc std: 0.00
Sample auc CI: 0.00
Confidence interval for AUC: [0.91 - 0.91]
0.4748187
              precision    recall  f1-score   support

         0.0       0.99      0.79      0.88     36808
         1.0       0.53      0.98      0.69      9026

    accuracy                           0.83     45834
   macro avg       0.76      0.89      0.79     45834
weighted avg       0.90      0.83      0.84     45834

0.5833080346987317


2647